# `f3dasm`: Framework for Data-Driven Design and Analysis of Structures and Materials 
*April 3rd, 2023* <br>
*Code Release Week \#1*

# Table of contents

**1. Project introduction**
- 1.1 Overview
- 1.2 Conceptual Map and use cases
- 1.3 Computational Framework
- 1.4 Installation and getting started

**2. Demonstration**

## 1.1 Overview

f3dasm is an attempt to unite data-driven design and analysis of structures of materials.
More concretely, 

## 1.2 Conceptual map and use cases

**Using `f3dasm` to handle your design of experiments**
- *Have your own functions and modules and coat them in a `f3dasm` sauce to manage and scale-up your experiments!*

**Using `f3dasm` to benchmark or compare models**
- *Go fully `f3dasm`: use existing implementations to benchmark parts of the data-driven machine learning process!*

**Develop on `f3dasm`**
- *Work hard, play hard: work towards making your implementations an official `f3dasm`extension!*



## 1.4 Installation and getting started

### System requirements
`f3dasm` is compatible with:
1. Python 3.7 to 3.10.
2. the three major operations system (Linux, MacOS, Ubuntu).
3. the default environment of Google Colab (Python 3.8, Linux) 
4. the `pip` package manager system.

Installation instruction can be found in the documentation page under [Getting Started](https://bessagroup.github.io/F3DASM/gettingstarted.html)

You can check if the installation was succesfull by importing f3dasm. It will show the installed version and the dependencies:

In [1]:
import f3dasm

2023-03-30 21:46:05,061 - Imported f3dasm (version: 0.2.98)
2023-03-30 21:46:05.862783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 21:46:06.019568: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-30 21:46:06.711568: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64:

#### Distinction between python package and repository
- The Python  PyPI package (`pip install f3dasm`) contains the code that is used when installing the package as a **user**. It contains only the `main` branch version.
- The GitHub repository is mainly for **developers** and besides the package includes:
  - Studies (more on that later)
  - Test suite
  - Documentation source
  - Tutorial notebooks

### Extensions

- The package contains a lot of implementation for each of the blocks.
- The installation of dependencies of `f3dasm` is modular: you decide what you want to use or not.

We can distinguish three ways of using `f3dasm`:

#### Using `f3dasm` to handle your design of experiments
*Have your own functions and modules and coat them in a `f3dasm` sauce to manage and scale-up your experiments!*

The **core** package: contains the minimal installation to use `f3dasm` without extended features. 
Installed with `pip install f3dasm`

The core package contains the following features:
1. provide a way to parametrize your experiment with the **design-of-experiments** classes.
2. provide the option to investigate their experiment by **sampling** and **optimizing** their design.
3. provide the user guidance in **parallelizing** their program and ordering their data.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The core package requires the following dependencies:
- `numpy` and `scipy`: for numerical operations
- `pandas`: for the representation of the design of experiments
- `matplotlib`: for plotting
- `hydra-core`: for deploying your experiment 
- `pathos`: for multiprocessing
- `autograd`: for computing gradients

#### Using `f3dasm` to benchmark or compare models
*Go fully `f3dasm`: use existing implementations to benchmark parts of the data-driven machine learning process!*

You can solely use the core package, but it is advised to enrich `f3dasm` with its **extensions** 

The extensions contain the following features:
1. provide various **implementations** to accommodate common machine learning workflows.
2. provide **adapter** classes that link common machine learning libraries to `f3dasm` base classes. 


For each of the blocks, extensions can be installed to extend the choice of implementations. Installed with `pip install f3dasm[<name of extension>]`

The following extensions are available:
- **machinelearning**: containing various `tensorflow` related models
- **sampling**: containing sampling strategies from `SALib`
- **optimization**: containing various optimizers from `GPyOpt`, `pygmo` and `tensorflow`


#### Develop on `f3dasm`
*Work hard, play hard: work towards making your implementations an official `f3dasm` extension!*

If you want your implementation to be part of the `f3dasm` package, you can develop an adapter and/or implementation for `f3dasm`

The **developement** package: contains the full installation plus requirements for developing on `f3dasm`. 
Installed with `pip install f3dasm[dev]`

Information on how to contribute to `f3dasm` can be found [on the wiki page of the GitHub repository](https://github.com/bessagroup/F3DASM/wiki)!

## 2. Demonstration

Import some other packages and set a seed

In [2]:
import numpy as np
import logging
from pathos.helpers import mp  # For multiprocessing!
import time # For ... timing!

SEED = 42

### Example: Set up your program with f3dasm

Let's say we have a program that we want to execute. It is important that this could be **anything**. Like:
- Calculate the loss of some compliance curve in topology optimization
- Computing the mean stress and strain from some abaqus simulation
- Benchmarking various regressors in a multi-fidelity setting

At the top level of your experiment, you will probably have a main function that accepts some arguments and returns the quantity of interest.

Let's create such a function, just for demonstration purposes.

In [3]:
def main(a: float, b: float, c: float) -> float:    
    functions = [f3dasm.functions.Rastrigin, f3dasm.functions.Levy, f3dasm.functions.Ackley]
    y = []
    for func in functions:
        f = func(dimensionality=3, scale_bounds=np.tile([-1.,1.], (3,1)))
        time.sleep(.1)
        y.append(f(np.array([a,b,c])).ravel()[0])

    # Sum the values
    out = sum(y)
    logging.info(f"Executed program with a={a:.3f}, b={b:.3f}, c={c:.3f}: \t Result {out:.3f}")
    return out

What are we seeing:
- The program requires three floating points and returns a float as well.
- It creates three 3D-benchmark functions, evaluates them sequentially and sums the results
- We simulate some computational cost (0.1 seconds per evaluation) by calling the `time.sleep()` method
- We write to a log

> Note: `my_own_program` uses the integrated benchmark functions from `f3dasm`, but this could very well be one of your codes without any dependency on `f3dasm`.

Executing multiple experiments is easy:

In [4]:
inputs = np.random.uniform(size=(10,3))

start_time = time.time()
outputs = np.array([main(*input_vals) for input_vals in inputs])
time_not_parallel = time.time() - start_time

print(f"It took {time_not_parallel:.5f} seconds to execute this for loop")

2023-03-30 21:46:08,348 - Executed program with a=0.630, b=0.084, c=0.420: 	 Result 137.605
2023-03-30 21:46:08,653 - Executed program with a=0.565, b=0.720, c=0.958: 	 Result 161.074
2023-03-30 21:46:08,958 - Executed program with a=0.186, b=0.529, c=0.109: 	 Result 54.651
2023-03-30 21:46:09,261 - Executed program with a=0.088, b=0.838, c=0.969: 	 Result 193.849
2023-03-30 21:46:09,566 - Executed program with a=0.850, b=0.437, c=0.872: 	 Result 153.952
2023-03-30 21:46:09,870 - Executed program with a=0.670, b=0.831, c=0.591: 	 Result 177.828
2023-03-30 21:46:10,175 - Executed program with a=0.529, b=0.785, c=0.642: 	 Result 147.754
2023-03-30 21:46:10,480 - Executed program with a=0.406, b=0.915, c=0.196: 	 Result 85.978
2023-03-30 21:46:10,784 - Executed program with a=0.290, b=0.636, c=0.783: 	 Result 113.106
2023-03-30 21:46:11,089 - Executed program with a=0.505, b=0.688, c=0.987: 	 Result 281.849


It took 3.04659 seconds to execute this for loop


We can save the values of `outputs` for later use

This process (`main.py`) can be described with the following figure:

<img src="sequential.png" alt="alt text" width="30%" height="30%">

### Local parallelization

If you are familiar with [multiprocessing](https://docs.python.org/3/library/multiprocessing.html), you might already know that we can speed-up this function by parellizing the internal for loop:

We create a multiprocessing pool (`mp.Pool()`) where we map the functions to cores in our machine:

In [5]:
def main_parallel(a: float, b: float, c: float) -> float:
    def evaluate_function(func, a, b, c):
        f = func(dimensionality=3, scale_bounds=np.tile([-1.,1.], (3,1)))
        y = f(np.array([a,b,c])).ravel()[0]
        time.sleep(.1)
        return y

    functions = [f3dasm.functions.Rastrigin, f3dasm.functions.Levy, f3dasm.functions.Ackley]
    with mp.Pool() as pool:
        y = pool.starmap(evaluate_function, [(func, a, b, c) for func in functions])

    # Sum the values
    out = sum(y)

    logging.info(f"Executed program with a={a:.3f}, b={b:.3f}, c={c:.3f}: \t Result: {out:.3f}")
    return out

Executing this function will speed up the process

In [6]:
inputs = np.random.uniform(size=(10,3))

start_time = time.time()
outputs = np.array([main_parallel(*input_vals) for input_vals in inputs])
time_parallel = time.time() - start_time

print(f"It took {time_parallel:.5f} seconds to execute this for loop")
print(f"We are {time_not_parallel-time_parallel:.5f} seconds faster by parellelization!")

2023-03-30 21:46:11,521 - Executed program with a=0.436, b=0.984, c=0.352: 	 Result: 172.199
2023-03-30 21:46:11,772 - Executed program with a=0.199, b=0.694, c=0.083: 	 Result: 109.216
2023-03-30 21:46:12,035 - Executed program with a=0.169, b=0.083, c=0.364: 	 Result: 115.599
2023-03-30 21:46:12,312 - Executed program with a=0.547, b=0.039, c=0.409: 	 Result: 173.194
2023-03-30 21:46:12,555 - Executed program with a=0.825, b=0.674, c=0.739: 	 Result: 260.838
2023-03-30 21:46:12,822 - Executed program with a=0.185, b=0.829, c=0.219: 	 Result: 106.291
2023-03-30 21:46:13,126 - Executed program with a=0.388, b=0.790, c=0.910: 	 Result: 242.613
2023-03-30 21:46:13,448 - Executed program with a=0.269, b=0.725, c=0.596: 	 Result: 163.385
2023-03-30 21:46:13,737 - Executed program with a=0.156, b=0.764, c=0.035: 	 Result: 102.934
2023-03-30 21:46:14,001 - Executed program with a=0.576, b=0.422, c=0.542: 	 Result: 165.176


It took 2.70007 seconds to execute this for loop
We are 0.34652 seconds faster by parellelization!


This process (`main_parallel.py`) can be described with the following figure:

<img src="parallel.png" alt="alt text" width="30%" height="30%">

### Scale-up: challenges

Now we would like to really scale things up. 

Q) What challenges lie along the way?

I asked ChatGPT:

- **1. Experiment design and analysis**: As the complexity of the experiment increases, it becomes more difficult to design experiments that are robust and reproducible, and to analyze the results in a meaningful way. This can lead to issues with experimental design, parameter tuning, and statistical analysis.

- **2. Parallelization and distribution**: As experiments become larger, it may be necessary to parallelize or distribute the computations across multiple machines or nodes in order to reduce the overall runtime. This introduces additional challenges such as synchronization between distributed processes.

- **3. Managing data**: As the volume of data generated by an experiment increases, it becomes more difficult to manage and store that data. This can lead to issues with data corruption, loss, or inconsistency.

This is where `f3dasm` is a helping hand!

#### 1. Experiment design and analysis

We can create a `f3dasm.DesignSpace` to capture the variables of interest:
- A `f3dasm.DesignSpace` consists of an input and output list of `f3dasm.Parameter` objects

In [7]:
param_a = f3dasm.ContinuousParameter(name='a', lower_bound=-1., upper_bound=1.)
param_b = f3dasm.ContinuousParameter(name='b', lower_bound=-1., upper_bound=1.)
param_c = f3dasm.ContinuousParameter(name='c', lower_bound=-1., upper_bound=1.)
param_out = f3dasm.ContinuousParameter(name='y')

design = f3dasm.DesignSpace(input_space=[param_a, param_b, param_c], output_space=[param_out])

We can create an object to store the experiments: `f3dasm.ExperimentData`, but we can also **sample from this designspace**
We do that with the `f3dasm.sampling` submodule:

> Note that this submodule offers an extension (`f3dasm[sampling]`) that include sampling strategies from `SALib` 

In [8]:
# Create the sampler object
sampler = f3dasm.sampling.RandomUniform(design=design, seed=SEED)

data: f3dasm.ExperimentData = sampler.get_samples(numsamples=10)

The data object is under the hood a pandas dataframe:

In [9]:
data.data

input                     output
          a         b         c      y
0 -0.250920  0.901429  0.463988    NaN
1  0.197317 -0.687963 -0.688011    NaN
2 -0.883833  0.732352  0.202230    NaN
3  0.416145 -0.958831  0.939820    NaN
4  0.664885 -0.575322 -0.636350    NaN
5 -0.633191 -0.391516  0.049513    NaN
6 -0.136110 -0.417542  0.223706    NaN
7 -0.721012 -0.415711 -0.267276    NaN
8 -0.087860  0.570352 -0.600652    NaN
9  0.028469  0.184829 -0.907099    NaN

The `y` values are NaN because we haven't evaluate our experiment yet! Let's do that:

Handy: we can retrieve the input columns of a specific row as a dictionary

In [10]:
data.get_inputdata_by_index(index=3)

{'a': 0.416145155592091, 'b': -0.9588310114083951, 'c': 0.9398197043239886}

Unpacking the values as arguments of our experiment creates the same results:

In [11]:
for index in range(data.get_number_of_datapoints()):
    value = main_parallel(**data.get_inputdata_by_index(index))
    data.set_outputdata_by_index(index, value)

2023-03-30 21:46:14,878 - Executed program with a=-0.251, b=0.901, c=0.464: 	 Result: 261.134
2023-03-30 21:46:15,111 - Executed program with a=0.197, b=-0.688, c=-0.688: 	 Result: 19.109
2023-03-30 21:46:15,324 - Executed program with a=-0.884, b=0.732, c=0.202: 	 Result: 321.825
2023-03-30 21:46:15,545 - Executed program with a=0.416, b=-0.959, c=0.940: 	 Result: 170.930
2023-03-30 21:46:15,807 - Executed program with a=0.665, b=-0.575, c=-0.636: 	 Result: 79.458
2023-03-30 21:46:16,067 - Executed program with a=-0.633, b=-0.392, c=0.050: 	 Result: 139.412
2023-03-30 21:46:16,334 - Executed program with a=-0.136, b=-0.418, c=0.224: 	 Result: 115.536
2023-03-30 21:46:16,599 - Executed program with a=-0.721, b=-0.416, c=-0.267: 	 Result: 83.109
2023-03-30 21:46:16,868 - Executed program with a=-0.088, b=0.570, c=-0.601: 	 Result: 215.214
2023-03-30 21:46:17,134 - Executed program with a=0.028, b=0.185, c=-0.907: 	 Result: 109.803


Now our data-object is filled

In [12]:
data.data

input                          output
          a         b         c           y
0 -0.250920  0.901429  0.463988  261.134214
1  0.197317 -0.687963 -0.688011   19.109039
2 -0.883833  0.732352  0.202230  321.825051
3  0.416145 -0.958831  0.939820  170.930424
4  0.664885 -0.575322 -0.636350   79.458296
5 -0.633191 -0.391516  0.049513  139.411721
6 -0.136110 -0.417542  0.223706  115.535908
7 -0.721012 -0.415711 -0.267276   83.109400
8 -0.087860  0.570352 -0.600652  215.214311
9  0.028469  0.184829 -0.907099  109.803282

This process can be described with the following figure:

<img src="single_node.png" alt="alt text" width="30%" height="30%">

#### 2. Paralellization and distribution

`f3dasm` can handle paralellization to the HPC cluster and experiment distribution. 

In order to set this up, navigate to a folder where you want to create your experiment and run f3dasm.experiment.quickstart()

In [13]:
# I'll not run this command because this is a demo

# f3dasm.experiment.quickstart()

This creates the following files and folders:

```
└── my_experiment 
    ├── main.py
    ├── config.py
    ├── config.yaml
    ├── default.yaml
    ├── pbsjob.sh
    └── README.md
    └── hydra/job_logging
        └── custom_script.py
```

Without going to much in detail, the following things have already been set up automatically:

**Logging**
- `hydra` (and the `custom_script.py`) take care of all (multiprocess) logging
- including writing across nodes when executing arrayjobs!

**Parameter storage**
- `config.yaml`, `config.py` and `default.yaml` can be used for easy reproducibility and parameter tuning of your experiment!

**Parallelization**
- `pbsjob.sh` can be used to execute your `main.py` file on the HPC, including array-jobs.

example:
```
qsub pbsjob.sh
qsub pbsjob.sh -t 0-10
```

**Saving data**
- `hydra` creates a new `outputs/<HPC JOBID>/` directory that saves all output files, logs and settings when executing `main.py`
- When executing arrayjobs, all arrayjobs write to the same folder!

#### 3. Managing data

Let's recall: our single node process with `f3dasm.ExperimentData` can be abstracted by the following image:

<img src="single_node.png" alt="alt text" width="30%" height="30%">

Parallelizing the **outer loop** is more difficult, but we can do that across nodes with help of the `f3dasm.experiment.JobQueue`


In [22]:
job_queue = f3dasm.experiment.JobQueue(filename='my_jobs')


{0: 'open', 1: 'open', 2: 'open', 3: 'open', 4: 'open', 5: 'open', 6: 'open', 7: 'open', 8: 'open', 9: 'open'}

We can fill the queue with the rows of the `f3dasm.ExperimentData` object:

In [23]:
job_queue.create_jobs_from_experimentdata(data)
job_queue

{0: 'open', 1: 'open', 2: 'open', 3: 'open', 4: 'open', 5: 'open', 6: 'open', 7: 'open', 8: 'open', 9: 'open'}

10 jobs have been added and they are all up for grabs!

Let's first write this to disk so multiple nodes can access it:

In [24]:
job_queue.write_new_jobfile()

A node can grab the first available job in the queue with the `get()` method:
The file is locked when accessing the information from the JSON file


In [26]:
job_id = job_queue.get()
print(f"The first open job_id is {job_id}!")

The first open job_id is 1!


After returning the `job_id`, the lock is removed and the job is changed to `in progress`

In [28]:
job_queue.get_jobs()

{0: 'in progress',
 1: 'in progress',
 2: 'open',
 3: 'open',
 4: 'open',
 5: 'open',
 6: 'open',
 7: 'open',
 8: 'open',
 9: 'open'}

When a new node asks a new job, it will return the next open job in line!

In [29]:
job_id = job_queue.get()
print(f"The first open job_id is {job_id}!")

The first open job_id is 2!


When a job is finished, you can mark it finished or with an error:

In [30]:
job_queue.mark_finished(index=0)
job_queue.mark_error(index=1)

job_queue.get_jobs()

{0: 'finished',
 1: 'error',
 2: 'in progress',
 3: 'open',
 4: 'open',
 5: 'open',
 6: 'open',
 7: 'open',
 8: 'open',
 9: 'open'}

We can now change our simple script to handle multiprocessing across nodes!

In [31]:
job_queue = f3dasm.experiment.JobQueue(filename='my_jobs2')
job_queue.create_jobs_from_experimentdata(data)

job_queue.write_new_jobfile()

while True:
    try:
        jobnumber = job_queue.get()
    except f3dasm.experiment.NoOpenJobsError:
        break
    
    args = data.get_inputdata_by_index(jobnumber)

    value = main_parallel(**data.get_inputdata_by_index(jobnumber))
    data.set_outputdata_by_index(jobnumber, value)

    job_queue.mark_finished(jobnumber)

data.data

2023-03-30 22:00:48,562 - Executed program with a=-0.251, b=0.901, c=0.464: 	 Result: 261.134
2023-03-30 22:00:48,816 - Executed program with a=0.197, b=-0.688, c=-0.688: 	 Result: 19.109
2023-03-30 22:00:49,102 - Executed program with a=-0.884, b=0.732, c=0.202: 	 Result: 321.825
2023-03-30 22:00:49,388 - Executed program with a=0.416, b=-0.959, c=0.940: 	 Result: 170.930
2023-03-30 22:00:49,666 - Executed program with a=0.665, b=-0.575, c=-0.636: 	 Result: 79.458
2023-03-30 22:00:49,952 - Executed program with a=-0.633, b=-0.392, c=0.050: 	 Result: 139.412
2023-03-30 22:00:50,230 - Executed program with a=-0.136, b=-0.418, c=0.224: 	 Result: 115.536
2023-03-30 22:00:50,555 - Executed program with a=-0.721, b=-0.416, c=-0.267: 	 Result: 83.109
2023-03-30 22:00:50,835 - Executed program with a=-0.088, b=0.570, c=-0.601: 	 Result: 215.214
2023-03-30 22:00:51,126 - Executed program with a=0.028, b=0.185, c=-0.907: 	 Result: 109.803


An unexpected error occurred: The jobfile my_jobs2 does not have any open jobs left!


input                          output
          a         b         c           y
0 -0.250920  0.901429  0.463988  261.134214
1  0.197317 -0.687963 -0.688011   19.109039
2 -0.883833  0.732352  0.202230  321.825051
3  0.416145 -0.958831  0.939820  170.930424
4  0.664885 -0.575322 -0.636350   79.458296
5 -0.633191 -0.391516  0.049513  139.411721
6 -0.136110 -0.417542  0.223706  115.535908
7 -0.721012 -0.415711 -0.267276   83.109400
8 -0.087860  0.570352 -0.600652  215.214311
9  0.028469  0.184829 -0.907099  109.803282

This process looks like this:

<img src="jobqueue.png" alt="alt text" width="30%" height="30%">